In [1]:
import os
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd 
import copy


import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import matplotlib.pyplot as plt

import sys 
from pathlib import Path
import plotly.express as px

module_path = ""
# file=os.path.join(module_path,"utils.py")
sys.path.append(module_path)
import utils as utils

import parse_effort_sizing as parse_effort
import parse_ongoing_resources as parse_ong
import resources_KPIs as res_kpis
from generic_resource_kpis import compute_utilization

# pipeline_tasks= {
#     ('Project1','Task1_1') : 
#         {'start_date':'1 JAN 2022','end_date':'31 JAN 2022',
#          'skillset':'AI_architect','uf':0.5} ,
    
#     ('Project1','Task2_1') : 
#         {'start_date':'1 FEB 2022','end_date':'28 FEB 2022',
#          'skillset':'BI','uf':1},
    
#     ('Project2','Task1_2') : 
#         {'start_date':'1 MAR 2022','end_date':'30 MAR 2022',
#          'skillset':'DS','uf':0.5} ,  # --- > For Kareem
    
#     ('Project2','Task2_2') : 
#         {'start_date':'1 MAR 2022','end_date':'31 MAR 2022',
#          'skillset':'DS','uf':1} ,
    
#     ('Project2','Task3_2') : 
#         {'start_date':'1 MAR 2022','end_date':'30 MAR 2022',
#          'skillset':'DS','uf':0.5} ,
#     ('Project3','Task1_3') : 
#         {'start_date':'5 MAR 2023','end_date':'7 MAR 2023',
#          'skillset':'BI','uf':1}  ,
    
#      ('Project3','Task2_3') : 
#         {'start_date':'1 JAN 2022','end_date':'5 JAN 2022',
#          'skillset':'DS','uf':0.5} , # --- > For Kareem

# }


# RESOURCES= {
#     'Amr' : 
#         {'skillset':'AI_architect','seniority': 'LEAD 2C','slots':1,
#         'current_load':{
#             'ong_1':{'project':'SWCC','start_date':'1 JAN 2022','end_date':'30 JUN 2022' ,'uf':0.5} ,
#             'ong_2':{'project':'MEWA','start_date':'1 JAN 2022','end_date':'31 MAR 2022' ,'uf':0.5}
#         }},
    
#     'Reem' : 
#         {'skillset':['BI','VIS'],"avail_start":'1 FEB 2022',"avail_end": '28 FEB 2022','slots':1,
#         'current_load':{
#             'ong_3':{'project':'SWCC','start_date':'1 JAN 2022','end_date':'30 JUN 2022' ,'uf':1} ,
#         }},
    
#     'Kareem' : 
#         {'skillset':'DS',"avail_start":'2 FEB 2022',"avail_end": '30 MAR 2023','slots':1,
#         'current_load':{
#             'ong_4':{'project':'SWCC','start_date':'1 JAN 2022','end_date':'30 JUN 2022' ,'uf':1} ,
#         }},
    
#     'Mohamed' : 
#         {'skillset':'BI',"avail_start":'1 MAR 2022',"avail_end": '30 MAR 2022','slots':1,
#         'current_load':{
#             'ong_5':{'project':'SWCC','start_date':'1 JAN 2022','end_date':'30 JUN 2022' ,'uf':1} ,
#         }},
    
#     'other1' :
#         {'skillset':'Devops',"avail_start":'1 JAN 2022',"avail_end": '31 JAN 2022','slots':1,
#         'current_load':{
#             'ong_6':{'project':'SWCC','start_date':'1 JAN 2022','end_date':'30 JUN 2022' ,'uf':1} ,
#         }}
 
# }


In [2]:
##Function to get the effort files in a list 
effort_sizing_list=[file for file in os.listdir("data") if "effort_sizing" in file ]
input_pipeline_list=[filename.split("_")[0] for filename in effort_sizing_list ]
print(effort_sizing_list)
print(input_pipeline_list)


['STC_effort_sizing.xlsx', 'TAKA_effort_sizing.xlsx', '~$TAKA_effort_sizing.xlsx']
['STC', 'TAKA', '~$TAKA']


In [3]:
##Take this as input 
parse_pipeline=["STC","TAKA"]   #This should be triggered by the GUI clicks

project_start_date_dict={"STC":"1 JAN 2023" , "TAKA":"15 MAR 2023"}

list_pipeline=[]
for project in parse_pipeline:
    # project_name="STC"
    # project_start_date="1 JAN 2023"
    path=f"data/{project}_effort_sizing.xlsx"
    project_start_date=project_start_date_dict[project]
    single_pipeline_tasks=parse_effort.main(project,project_start_date,path)
    list_pipeline.append(single_pipeline_tasks)

pipeline_tasks = {}
for d in list_pipeline:
    pipeline_tasks.update(d)

RESOURCES=parse_ong.main("data")
print(pipeline_tasks)
print("\n")
print(RESOURCES['Ibrahim Kareem'])

{('STC', 'P1_RHU_BDM'): {'start_date': '01 Jan 2023', 'end_date': '11 Feb 2023', 'skillset': 'BDM'}, ('STC', 'SC2_RHU_DRM'): {'start_date': '01 Jan 2023', 'end_date': '11 Feb 2023', 'skillset': 'DRM'}, ('STC', 'L1_RHU_AI Lead'): {'start_date': '07 Jan 2023', 'end_date': '19 Jan 2023', 'skillset': 'AI Lead'}, ('STC', 'L1_RHU_DG Lead'): {'start_date': '07 Jan 2023', 'end_date': '19 Jan 2023', 'skillset': 'DG Lead'}, ('STC', 'L1_RHU_DM Lead'): {'start_date': '07 Jan 2023', 'end_date': '19 Jan 2023', 'skillset': 'DM Lead'}, ('STC', 'C1_RHU_DM'): {'start_date': '19 Jan 2023', 'end_date': '08 Feb 2023', 'skillset': 'DM'}, ('STC', 'C1_AMM_DM'): {'start_date': '19 Jan 2023', 'end_date': '08 Feb 2023', 'skillset': 'DM'}, ('STC', 'C1_RHU_Coordinator'): {'start_date': '19 Jan 2023', 'end_date': '11 Feb 2023', 'skillset': 'Coordinator'}, ('TAKA', 'P1_RHU_BDM'): {'start_date': '15 Mar 2023', 'end_date': '22 Oct 2023', 'skillset': 'BDM'}, ('TAKA', 'SC2_RHU_DRM'): {'start_date': '15 Mar 2023', 'end_d

In [4]:
num_resources,bench_count,bench_list,resource_utilization_dict=res_kpis.main(RESOURCES)
overall_current_utilization=sum(resource_utilization_dict.values()) / len(resource_utilization_dict.keys())
print(f"You have {num_resources} Resources")
print(f"Currently there are {bench_count} on bench ")
print(f"The people on bench are {bench_list}")
# print("The resources utilization are ",resource_utilization_dict)
print("The overall utilization till year end is ",overall_current_utilization)

You have 13 Resources
Currently there are 1 on bench 
The people on bench are ['Mohamed unassigned']
The overall utilization till year end is  88.25


In [5]:
pd_1=utils.convert_to_pd(pipeline_tasks)
utils.plot_gantt_chart(pd_1)

In [6]:
ongoing_tasks=utils.resource_tasks(RESOURCES)
pipeline_tasks=utils.add_pipeline_type(pipeline_tasks)
TASKS=utils.merge_ongoing_pipeline(pipeline_tasks,ongoing_tasks)

all_tasks_overlap_tuples=utils.get_over_lap_tasks(TASKS)
all_task_overlap=utils.task_overlap_combination(all_tasks_overlap_tuples)

pipline_constraints,resource_constraints=utils.constraints_uf(all_task_overlap,TASKS)

new_resource_constraint=utils.reformulate_resource_constraints(resource_constraints,TASKS)

In [8]:
pipeline_project_tasks={}
for p,t in pipeline_tasks.keys():
    list_1=pipeline_project_tasks.get(p,[])
    list_1.append(t)
    pipeline_project_tasks[p]=list_1
print(pipeline_project_tasks)
print("\n")

pipeline_task_projects={}
for p,t in pipeline_tasks.keys():
    # print(p)
    # print(t)
    list_1=pipeline_project_tasks.get((p,t),[])
    list_1.append(p)
    pipeline_task_projects[(p,t)]=list_1
print(pipeline_task_projects)

{'STC': ['P1_RHU_BDM', 'SC2_RHU_DRM', 'L1_RHU_AI Lead', 'L1_RHU_DG Lead', 'L1_RHU_DM Lead', 'C1_RHU_DM', 'C1_AMM_DM', 'C1_RHU_Coordinator'], 'TAKA': ['P1_RHU_BDM', 'SC2_RHU_DRM', 'L1_RHU_AI Lead', 'L1_RHU_DG Lead', 'L1_RHU_DM Lead', 'C1_RHU_AI', 'C1_AMM_AI', 'C1_RHU_Coordinator']}


{('STC', 'P1_RHU_BDM'): ['STC'], ('STC', 'SC2_RHU_DRM'): ['STC'], ('STC', 'L1_RHU_AI Lead'): ['STC'], ('STC', 'L1_RHU_DG Lead'): ['STC'], ('STC', 'L1_RHU_DM Lead'): ['STC'], ('STC', 'C1_RHU_DM'): ['STC'], ('STC', 'C1_AMM_DM'): ['STC'], ('STC', 'C1_RHU_Coordinator'): ['STC'], ('TAKA', 'P1_RHU_BDM'): ['TAKA'], ('TAKA', 'SC2_RHU_DRM'): ['TAKA'], ('TAKA', 'L1_RHU_AI Lead'): ['TAKA'], ('TAKA', 'L1_RHU_DG Lead'): ['TAKA'], ('TAKA', 'L1_RHU_DM Lead'): ['TAKA'], ('TAKA', 'C1_RHU_AI'): ['TAKA'], ('TAKA', 'C1_AMM_AI'): ['TAKA'], ('TAKA', 'C1_RHU_Coordinator'): ['TAKA']}


In [14]:
pipeline_task_projects.keys()

dict_keys([('STC', 'P1_RHU_BDM'), ('STC', 'SC2_RHU_DRM'), ('STC', 'L1_RHU_AI Lead'), ('STC', 'L1_RHU_DG Lead'), ('STC', 'L1_RHU_DM Lead'), ('STC', 'C1_RHU_DM'), ('STC', 'C1_AMM_DM'), ('STC', 'C1_RHU_Coordinator'), ('TAKA', 'P1_RHU_BDM'), ('TAKA', 'SC2_RHU_DRM'), ('TAKA', 'L1_RHU_AI Lead'), ('TAKA', 'L1_RHU_DG Lead'), ('TAKA', 'L1_RHU_DM Lead'), ('TAKA', 'C1_RHU_AI'), ('TAKA', 'C1_AMM_AI'), ('TAKA', 'C1_RHU_Coordinator')])

In [32]:
pipeline_tasks.keys()

dict_keys([('STC', 'P1_RHU_BDM'), ('STC', 'SC2_RHU_DRM'), ('STC', 'L1_RHU_AI Lead'), ('STC', 'L1_RHU_DG Lead'), ('STC', 'L1_RHU_DM Lead'), ('STC', 'C1_RHU_DM'), ('STC', 'C1_AMM_DM'), ('STC', 'C1_RHU_Coordinator'), ('TAKA', 'P1_RHU_BDM'), ('TAKA', 'SC2_RHU_DRM'), ('TAKA', 'L1_RHU_AI Lead'), ('TAKA', 'L1_RHU_DG Lead'), ('TAKA', 'L1_RHU_DM Lead'), ('TAKA', 'C1_RHU_AI'), ('TAKA', 'C1_AMM_AI'), ('TAKA', 'C1_RHU_Coordinator')])

In [11]:
model = ConcreteModel()
model.pipeline_tasks = Set(initialize = pipeline_tasks.keys(), dimen=2)
# model.pipeline_taskids_only=Set(initialize=list(map(lambda x : x[1],pipeline_tasks.keys())),dimen=1)
model.pipeline_taskids_only=Set(initialize=pipeline_task_projects.keys(),dimen=1)


model.pipeline_projects = Set(initialize = list(set([j for (j,m) in model.pipeline_tasks])))

model.RESOURCES =  Set(initialize = RESOURCES.keys(), dimen=1)

model.Resource_assignment=Set(initialize=model.pipeline_tasks * model.RESOURCES, dimen=3)

# model.Resource_assignment_proper=     Set(
#         initialize=model.pipeline_tasks * model.RESOURCES, dimen=3 ,
#        filter = lambda model,p,t,r : TASKS[(p,t)]['skillset'] in RESOURCES[r]['skillset']
#                 )
# print("\n")
# print(model.Resource_assignment.value_list)

    action taken
    action taken
    no action taken
    no action taken
    no action taken
    pipeline_taskids_only; no action taken


In [9]:
model.prt_improper_var=Var(model.Resource_assignment,within=Binary, initialize=0)

model.ra_var=Var(model.RESOURCES,within=NonNegativeReals, initialize=0)


model.ta_var=Var(model.pipeline_taskids_only,within=Binary, initialize=0)

In [29]:
pipeline_task_projects

{('STC', 'P1_RHU_BDM'): ['STC'],
 ('STC', 'SC2_RHU_DRM'): ['STC'],
 ('STC', 'L1_RHU_AI Lead'): ['STC'],
 ('STC', 'L1_RHU_DG Lead'): ['STC'],
 ('STC', 'L1_RHU_DM Lead'): ['STC'],
 ('STC', 'C1_RHU_DM'): ['STC'],
 ('STC', 'C1_AMM_DM'): ['STC'],
 ('STC', 'C1_RHU_Coordinator'): ['STC'],
 ('TAKA', 'P1_RHU_BDM'): ['TAKA'],
 ('TAKA', 'SC2_RHU_DRM'): ['TAKA'],
 ('TAKA', 'L1_RHU_AI Lead'): ['TAKA'],
 ('TAKA', 'L1_RHU_DG Lead'): ['TAKA'],
 ('TAKA', 'L1_RHU_DM Lead'): ['TAKA'],
 ('TAKA', 'C1_RHU_AI'): ['TAKA'],
 ('TAKA', 'C1_AMM_AI'): ['TAKA'],
 ('TAKA', 'C1_RHU_Coordinator'): ['TAKA']}

In [30]:
model.pipeline_taskids_only.value_list

    .ordered_data() to retrieve the values from a finite set in a
    deterministic order.  (deprecated in 5.7) (called from
    c:\jupyter\Anaconda3\envs\mo\lib\site-
    packages\IPython\core\interactiveshell.py:3398)


['P1_RHU_BDM',
 'SC2_RHU_DRM',
 'L1_RHU_AI Lead',
 'L1_RHU_DG Lead',
 'L1_RHU_DM Lead',
 'C1_RHU_DM',
 'C1_AMM_DM',
 'C1_RHU_Coordinator',
 'C1_RHU_AI',
 'C1_AMM_AI']

In [10]:
#CONSTRAINT FOR MAPPING TWO vars together (resources & Resource assignment)

model.cons = ConstraintList()
for resource in model.RESOURCES :
#     print(resource)
    model.cons.add(
    sum(model.prt_improper_var[p,t,resource] for p in model.pipeline_projects for t in pipeline_project_tasks[p]  ) ==
    model.ra_var[resource] 
    )

for task in model.pipeline_taskids_only:
    model.cons.add(
    sum(model.prt_improper_var[p,task,r] for p in pipeline_task_projects[task] for r in model.RESOURCES  ) ==
    model.ta_var[task] 
    )

In [11]:
#Any given task should have one or zero assignment
for t in model.pipeline_taskids_only:  
        model.cons.add(
                1 >= sum(  model.prt_improper_var[p,t,r] 
                         for r in model.RESOURCES 
                         for p in pipeline_task_projects[t])
            )

#The role required skillset must be in the resources skillset 
for p in model.pipeline_projects:
    for t in pipeline_project_tasks[p]:
        for r in model.RESOURCES :                         
            if  TASKS[(p,t)]['skillset'] not in RESOURCES[r]['skillset']:
                model.cons.add(
                        0== model.prt_improper_var[p,t,r]
                        )

In [12]:
#For any overlap tasks the sum of utiliztion factor should be less than or equal 
for r in model.RESOURCES:
        

    # pipleine constraints (can apply to any resource of the specific skillset)       
    for pipline_constraint in pipline_constraints :        
        #if RESOURCES[r]['skillset']==TASKS[pipline_constraint[0]]['skillset'] :   
        # if TASKS[pipline_constraint[0]]['skillset'] in RESOURCES[r]['skillset']  :         
            
        
        model.cons.add(
                RESOURCES[r]['slots'] >= sum( TASKS[proj_task].get('uf',1) * model.prt_improper_var[proj_task[0],proj_task[1],r] 
                    for proj_task in pipline_constraint )
        )
            
            
    # resource constraints (specific to this resource)  
        
    for resource_constraint in new_resource_constraint.get(r,[]) : 
        
        model.cons.add(        
             RESOURCES[r]['slots'] >= sum( TASKS[proj_task].get('uf',1) * model.prt_improper_var[proj_task[0],proj_task[1],r] 
                     for proj_task in resource_constraint['pipeline'])
            
                  + sum(TASKS[proj_task].get('uf',1) for proj_task in resource_constraint['ongoing'])
                    )


In [13]:
expr=20 * sum(model.ra_var[r] for r in model.RESOURCES) + 4 * sum(model.ta_var[t] for t in model.pipeline_taskids_only)
model.objective=Objective(expr=expr,sense=maximize)

In [14]:
os.environ['NEOS_EMAIL'] = 'amr.mansour@devoteam.com'
opt = SolverFactory('cbc')  # Select solver
solver_manager = SolverManagerFactory('neos')  # Solve in neos server
results = solver_manager.solve(model, opt=opt)

In [15]:
def get_resource_assignment(resource_assignment):
    resource_table = {
        p: {t: [] for t in pipeline_project_tasks[p]} 
        for p in model.pipeline_projects}
    
    
    for p in model.pipeline_projects:
        for t in pipeline_project_tasks[p]:
            for r in model.RESOURCES :
                    if model.prt_improper_var[p, t, r].value == 1:
                        resource_table[p][t].append(r)
    return resource_table

ra_dict=get_resource_assignment(model.prt_improper_var)
ra_dict

{'TAKA': {'P1_RHU_BDM': ['Saleh Samaneh'],
  'SC2_RHU_DRM': ['Waleed Abaas'],
  'L1_RHU_AI Lead': ['Amr Mansour'],
  'L1_RHU_DG Lead': [],
  'L1_RHU_DM Lead': ['Saleh Samaneh'],
  'C1_RHU_AI': ['Mohamed Abo Saad'],
  'C1_AMM_AI': ['Hawraa'],
  'C1_RHU_Coordinator': ['Ibrahim Kareem']},
 'STC': {'P1_RHU_BDM': ['Saleh Samaneh'],
  'SC2_RHU_DRM': ['Waleed Abaas'],
  'L1_RHU_AI Lead': ['Amr Mansour'],
  'L1_RHU_DG Lead': [],
  'L1_RHU_DM Lead': ['Saleh Samaneh'],
  'C1_RHU_DM': ['Mohamed Qasim'],
  'C1_AMM_DM': ['Mohamed Qasim'],
  'C1_RHU_Coordinator': ['Ibrahim Kareem']}}

In [16]:
def assignment_fullfilment(project,task):
    dict_1=ra_dict[project]
    try :
        assignment=dict_1[task][0]
    except: 
        assignment="To be Hired"
    return assignment

pd_2=utils.convert_to_pd(pipeline_tasks)
pd_2['assignment_status']=pd_2[["Project","Task"]].apply(lambda x:assignment_fullfilment(*x),axis=1)

utils.plot_assignment_gantt_chart(pd_2)

In [17]:
RESOURCES_future=copy.deepcopy(RESOURCES)

i=0
for resource in RESOURCES_future.keys():
    # RESOURCES_future[resource]['num_ongoing_and_future_projects']=RESOURCES_future[resource]['num_ongoing_projects']
    # RESOURCES_future[resource]['current_and_future_load']=RESOURCES_future[resource]['current_load']

    for p in model.pipeline_projects:
            for t in pipeline_project_tasks[p]:
                if model.prt_improper_var[p, t, resource].value == 1:
                    i=i+1
                    start_date=pipeline_tasks[(p,t)]['start_date']
                    end_date=pipeline_tasks[(p,t)]['end_date']
                    RESOURCES_future[resource]['num_ongoing_projects']=RESOURCES_future[resource]['num_ongoing_projects']+1
                    RESOURCES_future[resource]['current_load'][f'fut_{i}']={
                            'project': (p,t), 'start_date': start_date, 'end_date':end_date,'uf':1}                    

    

In [18]:
dict_assignment_resource_util=compute_utilization("1 JAN 2023","31 DEC 2023",RESOURCES_future)
dict_assignment_resource_util

{'Saleh Samaneh': 60,
 'Mohamed Hatab': 100,
 'Mohamed Qasim': 5,
 'Amr Mansour': 3,
 'Gayda Mtuhar': 100,
 'Ahmad Cheble': 0,
 'Mohamed Abo Saad': 55,
 'Ibrahim Kareem': 100,
 'Hawraa': 55,
 'Waleed Abaas': 60,
 'Amr Ramadan': 100,
 'Akram': 100,
 'Mohamed unassigned': 0}

In [19]:
overall_future_utilization=sum(dict_assignment_resource_util.values()) / len(dict_assignment_resource_util.keys())
# print(f"You have {num_resources} Resources")
# print(f"Currently there are {bench_count} on bench ")
# print(f"The people on bench are {bench_list}")
# print("The resources utilization are ",resource_utilization_dict)
print("The forseen utilization is ",overall_future_utilization)

The forseen utilization is  56.76923076923077


In [20]:
pd_2

,Project,Task,Start,End,Role,Type,assignment_status
0,TAKA,P1_RHU_BDM,2023-01-01,2023-02-11,BDM,pipeline,Saleh Samaneh
1,TAKA,SC2_RHU_DRM,2023-01-01,2023-02-11,DRM,pipeline,Waleed Abaas
2,TAKA,L1_RHU_AI Lead,2023-01-07,2023-01-19,AI Lead,pipeline,Amr Mansour
3,TAKA,L1_RHU_DG Lead,2023-01-07,2023-01-19,DG Lead,pipeline,To be Hired
4,TAKA,L1_RHU_DM Lead,2023-01-07,2023-01-19,DM Lead,pipeline,Saleh Samaneh
5,TAKA,C1_RHU_DM,2023-01-19,2023-02-08,DM,pipeline,To be Hired
6,TAKA,C1_AMM_DM,2023-01-19,2023-02-08,DM,pipeline,To be Hired
7,TAKA,C1_RHU_Coordinator,2023-01-19,2023-02-11,Coordinator,pipeline,Ibrahim Kareem
8,TAKA,P1_RHU_BDM,2023-03-15,2023-10-22,BDM,pipeline,Saleh Samaneh
9,TAKA,SC2_RHU_DRM,2023-03-15,2023-10-22,DRM,pipeline,Waleed Abaas


In [22]:
#Hiring Status 
# You need to hire the following
print("You need to hire the following : ")
pd_2[pd_2['assignment_status']=="To be Hired"]['Task']

,Project,Task,Start,End,Role,Type,assignment_status
3,TAKA,L1_RHU_DG Lead,2023-01-07,2023-01-19,DG Lead,pipeline,To be Hired
5,TAKA,C1_RHU_DM,2023-01-19,2023-02-08,DM,pipeline,To be Hired
6,TAKA,C1_AMM_DM,2023-01-19,2023-02-08,DM,pipeline,To be Hired
11,TAKA,L1_RHU_DG Lead,2023-03-21,2023-04-02,DG Lead,pipeline,To be Hired


In [23]:
pd_2

,Project,Task,Start,End,Role,Type,assignment_status
0,TAKA,P1_RHU_BDM,2023-01-01,2023-02-11,BDM,pipeline,Saleh Samaneh
1,TAKA,SC2_RHU_DRM,2023-01-01,2023-02-11,DRM,pipeline,Waleed Abaas
2,TAKA,L1_RHU_AI Lead,2023-01-07,2023-01-19,AI Lead,pipeline,Amr Mansour
3,TAKA,L1_RHU_DG Lead,2023-01-07,2023-01-19,DG Lead,pipeline,To be Hired
4,TAKA,L1_RHU_DM Lead,2023-01-07,2023-01-19,DM Lead,pipeline,Saleh Samaneh
5,TAKA,C1_RHU_DM,2023-01-19,2023-02-08,DM,pipeline,To be Hired
6,TAKA,C1_AMM_DM,2023-01-19,2023-02-08,DM,pipeline,To be Hired
7,TAKA,C1_RHU_Coordinator,2023-01-19,2023-02-11,Coordinator,pipeline,Ibrahim Kareem
8,TAKA,P1_RHU_BDM,2023-03-15,2023-10-22,BDM,pipeline,Saleh Samaneh
9,TAKA,SC2_RHU_DRM,2023-03-15,2023-10-22,DRM,pipeline,Waleed Abaas


WorkSpace

In [19]:
RESOURCES_future['Saleh Samaneh']

{'skillset': ['DQ Lead', 'DM Lead', 'BDM'],
 'seniority': 'P1',
 'slots': 3,
 'hire_date': Timestamp('2021-01-01 00:00:00'),
 'num_ongoing_projects': 6,
 'current_load': {'ong_0': {'project': 'MOJ',
   'start_date': '01 Jan 2022',
   'end_date': '31 Dec 2022',
   'uf': 1},
  'ong_1': {'project': 'MOE',
   'start_date': '01 Jan 2022',
   'end_date': '31 Dec 2022',
   'uf': 1},
  'fut_1': {'project': ('STC', 'P1_RHU_BDM'),
   'start_date': '01 Jan 2023',
   'end_date': '11 Feb 2023',
   'uf': 1},
  'fut_2': {'project': ('STC', 'L1_RHU_DM Lead'),
   'start_date': '07 Jan 2023',
   'end_date': '19 Jan 2023',
   'uf': 1},
  'fut_3': {'project': ('TAKA', 'P1_RHU_BDM'),
   'start_date': '15 Mar 2023',
   'end_date': '22 Oct 2023',
   'uf': 1},
  'fut_4': {'project': ('TAKA', 'L1_RHU_DM Lead'),
   'start_date': '21 Mar 2023',
   'end_date': '02 Apr 2023',
   'uf': 1}}}

In [20]:
all_projects_dictionary=RESOURCES_future['Mohamed unassigned']['current_load']
all_projects_dictionary

{'ong_12': {'project': 'idle',
  'start_date': '01 JAN 1900',
  'end_date': '02 JAN 1900',
  'uf': 0}}

In [21]:
all_projects_dictionary['ong_12']['project']

'idle'

In [22]:
def get_resource_assignment(resource_assignment):
    resource_table = {
        
        p: {t: [] for t in pipeline_project_tasks[p]} 
        for p in model.pipeline_projects}   
    
    
    for r in model.RESOURCES :
        for p in model.pipeline_projects:
            for t in pipeline_project_tasks[p]:
                if model.prt_improper_var[p, t, r].value == 1:
                    resource_table[p][t].append(r)
    return resource_table

get_resource_assignment(model.prt_improper_var)

{'STC': {'P1_RHU_BDM': ['Saleh Samaneh'],
  'SC2_RHU_DRM': ['Waleed Abaas'],
  'L1_RHU_AI Lead': ['Amr Mansour'],
  'L1_RHU_DG Lead': [],
  'L1_RHU_DM Lead': ['Saleh Samaneh'],
  'C1_RHU_DM': ['Mohamed Qasim'],
  'C1_AMM_DM': ['Mohamed Qasim'],
  'C1_RHU_Coordinator': ['Ibrahim Kareem']},
 'TAKA': {'P1_RHU_BDM': ['Saleh Samaneh'],
  'SC2_RHU_DRM': ['Waleed Abaas'],
  'L1_RHU_AI Lead': ['Amr Mansour'],
  'L1_RHU_DG Lead': [],
  'L1_RHU_DM Lead': ['Saleh Samaneh'],
  'C1_RHU_AI': ['Mohamed Abo Saad'],
  'C1_AMM_AI': ['Hawraa'],
  'C1_RHU_Coordinator': ['Ibrahim Kareem']}}

In [23]:
RESOURCES['Saleh Samaneh']

{'skillset': ['DQ Lead', 'DM Lead', 'BDM'],
 'seniority': 'P1',
 'slots': 3,
 'hire_date': Timestamp('2021-01-01 00:00:00'),
 'num_ongoing_projects': 2,
 'current_load': {'ong_0': {'project': 'MOJ',
   'start_date': '01 Jan 2022',
   'end_date': '31 Dec 2022',
   'uf': 1},
  'ong_1': {'project': 'MOE',
   'start_date': '01 Jan 2022',
   'end_date': '31 Dec 2022',
   'uf': 1}}}

In [24]:
RESOURCES

{'Saleh Samaneh': {'skillset': ['DQ Lead', 'DM Lead', 'BDM'],
  'seniority': 'P1',
  'slots': 3,
  'hire_date': Timestamp('2021-01-01 00:00:00'),
  'num_ongoing_projects': 2,
  'current_load': {'ong_0': {'project': 'MOJ',
    'start_date': '01 Jan 2022',
    'end_date': '31 Dec 2022',
    'uf': 1},
   'ong_1': {'project': 'MOE',
    'start_date': '01 Jan 2022',
    'end_date': '31 Dec 2022',
    'uf': 1}}},
 'Mohamed Hatab': {'skillset': ['DM', 'DQ'],
  'seniority': 'P1',
  'slots': 3,
  'hire_date': Timestamp('2021-01-01 00:00:00'),
  'num_ongoing_projects': 1,
  'current_load': {'ong_2': {'project': 'HRSD',
    'start_date': '01 Jan 2022',
    'end_date': '01 Jan 2024',
    'uf': 1}}},
 'Mohamed Qasim': {'skillset': ['DM', 'DQ'],
  'seniority': 'SC2',
  'slots': 2,
  'hire_date': Timestamp('2021-01-01 00:00:00'),
  'num_ongoing_projects': 1,
  'current_load': {'ong_3': {'project': 'MEWA',
    'start_date': '01 Jan 2022',
    'end_date': '31 Dec 2022',
    'uf': 1}}},
 'Amr Mansour': 